In [1]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Initial imports
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("listings.csv")
df.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,183319,https://www.airbnb.com/rooms/183319,20220909140055,2022-09-10,previous scrape,Panoramic Ocean View Venice Beach,Craftsmen style penthouse with ocean view from...,About as close to the beach as you can live in...,https://a0.muscache.com/pictures/1170205/e2596...,867995,...,5.00,5.00,5.00,NaN,f,2,2,0,0,0.02
1,51307,https://www.airbnb.com/rooms/51307,20220909140055,2022-09-10,city scrape,Spanish Bungalow Guest House LA CA. 30 plus ni...,PRIVATE GUEST HOUSE<br /><br /><b>The space</b...,"Local LA Community , shops and restaurants in...",https://a0.muscache.com/pictures/294658/ac8d6e...,235568,...,4.85,4.95,4.76,NaN,f,2,2,0,0,0.98
2,184314,https://www.airbnb.com/rooms/184314,20220909140055,2022-09-10,previous scrape,Boho Chic Flat..Steps to Beach!,"Bright, airy, quiet 1 bdr located just steps f...",You are in one of the beach locations in Los A...,https://a0.muscache.com/pictures/0bc2f8b7-9378...,884031,...,4.93,4.89,4.85,NaN,t,1,1,0,0,0.22


In [3]:
# Extract prices from listingDF into priceDF
priceDF = df['price']
prices=[]

# Convert prices from listingDF into float values and append it in prices list
for p in priceDF:
    p = float(p[1:].replace(',',''))
    prices.append(p)
    
# Replace the price column in the original listingDF with the new prices
df['price'] = prices

In [4]:
#Convert NaN to 0
df.fillna(0, inplace=True)

In [5]:
# separating dataset
airbnb_features = ["latitude", "longitude", "accommodates", "bedrooms", "beds", "minimum_nights", "maximum_nights", "number_of_reviews", "review_scores_rating", ]


y = df.price 
X = df[airbnb_features]

X.head()

,latitude,longitude,accommodates,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating
0,33.99211,-118.47600,2,2.0,2.0,30,90,3,5.00
1,34.12206,-118.26783,2,1.0,1.0,30,365,138,4.64
2,33.97487,-118.46312,1,1.0,1.0,30,30,30,4.88
3,34.00389,-118.44126,2,0.0,1.0,3,365,378,4.73
4,33.98301,-118.38607,6,2.0,3.0,30,730,2,4.00


In [6]:
y.head()

0    152.0
1     75.0
2    125.0
3    189.0
4    115.0
Name: price, dtype: float64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [8]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(34361, 9)
(11454, 9)
(34361,)
(11454,)


In [9]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeRegressor()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [11]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)
predictions

array([ 79., 352., 520., ..., 150., 278., 106.])

In [12]:
#Report the score of the prediction using the testing set
from sklearn.model_selection import cross_val_score


#score = cross_val_score(regressor, X_test, y_test)
score = regressor.score(X_test, y_test)

print(score)

NameError: name 'regressor' is not defined

In [ ]:
# this function takes both the training and validation sets to compute the MAE for a Decision Tree
def compute_mae(train_X, train_Y, validation_X, validation_Y, max_leaf_nodes):
    trees_model = DecisionTreeRegressor(max_leaf_nodes = max_leaf_nodes, random_state = 81)
    trees_model.fit(train_X, train_Y)
    validation_predictions = trees_model.predict(validation_X)
    error = mean_absolute_error(validation_Y, validation_predictions)
    
    return(error)

def get_best_tree_size(train_X, train_Y, validation_X, validation_Y, verbose = False):
    # candidates to iterate on finding a better tree depth
    candidate_max_leaf_nodes = [5, 10, 20, 30, 50, 100, 250, 500]

    # initialization
    minimum_error = None
    best_tree_size = 5

    # loop to find the minimal error value
    for max_leaf_nodes in candidate_max_leaf_nodes:
        current_error = compute_mae(train_X, train_Y, validation_X, validation_Y, max_leaf_nodes)
        verbose and print("(Size: {}, MAE: {})".format(max_leaf_nodes, current_error))

        if(minimum_error == None or current_error < minimum_error):
            minimum_error = current_error
            best_tree_size = max_leaf_nodes

    return(best_tree_size)

best_tree_size = get_best_tree_size(train_X, train_Y, validation_X, validation_Y, True)
best_tree_size

In [ ]:
# create the model
lisbon_airbnb_model = DecisionTreeRegressor(max_leaf_nodes = best_tree_size, random_state = 81)
lisbon_airbnb_model.fit(train_X, train_Y)

# generate the predictions for the validation set
validation_predictions = lisbon_airbnb_model.predict(validation_X)
validation_prediction_errors = mean_absolute_error(validation_Y, validation_predictions)

validation_prediction_errors

In [ ]:
# separating dataset, adding categorical
airbnb_features_ext = ["room_type", "latitude", "longitude", "accommodates", "bedrooms", "beds", "minimum_nights", "maximum_nights", "number_of_reviews", "review_scores_rating", ]

X = df[airbnb_features_ext]
X.head()

In [ ]:
#extracting categorical
categorical = (X.dtypes == 'object')
categorial_columns = list(categorical[categorical].index)

categorial_columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

train_X, validation_X, train_Y, validation_Y = train_test_split(X, Y, random_state = 81)

# create copies of our data sets to apply the transformations
train_X_encoded = train_X.copy()
validation_X_encoded = validation_X.copy()

encoder_new = LabelEncoder()

# for each column we fit the encoder and transform each of the rows
for column in categorial_columns:
    train_X_encoded[column] = encoder_new.fit_transform(train_X[column])
    validation_X_encoded[column] = encoder_new.transform(validation_X[column])

# a sample of our transformed data
train_X_encoded.head()


In [ ]:
train_X_encoded.to_numpy()

In [ ]:
# compute the best tree size
best_tree_size = get_best_tree_size(train_X_encoded, train_Y, validation_X_encoded, validation_Y)

# create the model
airbnb_model = DecisionTreeRegressor(max_leaf_nodes = best_tree_size, random_state = 81)
airbnb_model.fit(train_X_encoded, train_Y)

# generate the predictions for the validation set
validation_predictions = airbnb_model.predict(validation_X_encoded)
validation_prediction_errors = mean_absolute_error(validation_Y, validation_predictions)

validation_prediction_errors

In [ ]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(train_X_encoded.values)

# Scaling the data.
X_train_scaled = X_scaler.transform(train_X_encoded.values)
X_test_scaled = X_scaler.transform(validation_X_encoded.values)

In [ ]:
# Making predictions using the testing data.
predictions = airbnb_model.predict(X_test_scaled)